In [3]:
import numpy
import matplotlib.pyplot as plt
import pandas as pd
import math
import matplotlib
import datetime
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
matplotlib.style.use('ggplot')
%matplotlib inline

pd.set_option('precision',8)

Using TensorFlow backend.


In [4]:
# create date parser
def dateparse(time_in_secs):    
    return datetime.datetime.fromtimestamp(float(time_in_secs))

In [5]:
# Load dataset
potts = pd.read_csv('../../bld/out/data_raw/BTC_POT_chart_data.csv', parse_dates=True, index_col='date', date_parser=dateparse)

In [6]:
# Remove duplicated rows
df = potts[~ potts.index.duplicated()]

# cutting dataset down to usable size
# comment for full DS
df = df.iloc[0:2000,:]



KeyError: 'BTC_POT_AMOUNT'

In [30]:
# ensure float32 type
ds = df.values
ds = ds.astype('float32')

TypeError: float() argument must be a string or a number, not 'Timedelta'

In [ ]:
# rescale dataset values to be within 0 and 1
scaler = MinMaxScaler(feature_range=(0,1))
dataset = scaler.fit_transform(dataset)
# rescale whole dataset -> need to comensate for later

In [ ]:
# split into train and test sets# cannot use crossvalidation because of timeseries
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [ ]:
# convert an array of values into a dataset matrix
def create_features(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [ ]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))